In [1]:
import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
import numpy as np

(x_set, y_set), (x_test, y_test) = mnist.load_data()
x_set = x_set/255

c:\users\max\desktop\env\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Introduction

Given we will not be having class next week and I cannot reasonably expect you to do work for which we will not have lectured; this weeks sprint will be broken up into two smaller pieces as was lossely voted on in class, with this being part 1.

For this sprint you will be doing a process called K-Fold Cross Validation.

### Instructions

In class you were briefly introduced to Keras, which is a high level machine learning library that can be used to create everything from an introductory model such as what you will be building to very complex models used in industry every day to handle everything from chat bots to object detection and more.

### Section 1

In the last sprint you did some exploration that helped you understand the dataset and what was in it, this time you are going to prepare it for training. 

Professor Memon had talked about in his lecture taking your data and properly holding back some of it so that later you could use it to validate if your model was working or not.

For this section you will be responsible for implementing in python an algorithm called K-Fold

This will be worth **40** points of the sprint


### Section 2

With K = 5 for the number of folds you will do the below:

Now that you have properly segmented your data you will have to train K-1 models and validate them. The code for the model has already been implemented, you do not need to worry about that.

The general procedure is:
    1. Split your dataset into K even sets of data using the k-fold algorithm.
    2. Train a model on set K=0
    3. Validate the model on set K=1
    4. Repeat for K+1 and K+2
    
**Note:** Training the models will take some time depending on your computer, each model will be saved so after you are sure this part is working you should only have to do it once. If you mess something up you can delete the model files and start again.
    
This will be worth **40** points of the sprint

### Section 3
Provide a few sentences about common pitfalls of k-fold-cross validation and training models with it.

This will be worth **20** points of the sprint

### Extra credit

There are very many other validation methods for constructing machine learning models. Find one and implement it.
This is worth **20** extra credit points for the sprint.


#### Note:
Before you begin, you can use the same virtual environments you created last week, but you must pip install h5py into them. h5py is a file format library that will be used to save the trained models. 



In [2]:
def k_fold_split(x_set, y_set, folds=1):
    '''
    Inputs: The x_set data from mnist, the y_set labels from mnist
    Expected Output: The shuffled and K split datasets
    '''
    print("within k fold split function")
    np.random.seed(1)
    fold_size = int(len(x_set) / folds)
    
    x_set_temp = np.reshape(x_set, (60000, 784))
    
    print("x_set type is" + str(type(x_set)))
    print("y_set type is" + str(type(y_set)))
    print("X set is " + str(len(x_set)))
    print("Y set is " + str(len(y_set)))
    
    combined_list = [(x_set_temp[i], y_set[i]) for i in range(len(x_set))]
    np.random.shuffle(combined_list)
    
    five_lists = [combined_list[i::folds] for i in range(folds)]
    
    #one list that holds 5 lists
    x_final_list = [tup[0] for lst in five_lists for tup in lst]
    x_final_list = [x_final_list[i::folds] for i in range(folds)]
    for i in range(len(x_final_list)):
        x_final_list[i] = np.array(x_final_list[i])
    
    y_final_list = [tup[1] for lst in five_lists for tup in lst]
    y_final_list = [y_final_list[i::folds] for i in range(folds)]
    #each item is a numpy array with shape (12000, 784)
    #for i in range(len(y_final_list)):
        #y_final_list[i] = np.array(y_final_list[i])
    
    print("len of x final list " + str(len(x_final_list)))
    print("len of y final list " + str(len(y_final_list)))
    print("x_final_type " + str(type(x_final_list)))
    print("y_final_type " + str(type(y_final_list)))
    print("x_final_type[0] " + str(type(x_final_list[0])))
    print("y_final_type[0] " + str(type(y_final_list[0])))
    print("length of y final[0] " + str(len(y_final_list[0])))
    print("type of y final[0] " + str(type(y_final_list[0])))
    print("x_final_type[0] shape " + str(x_final_list[0].shape))
    #print("y_final_type shape" + str(y_final_list[0].shape))
    
    return x_final_list, y_final_list
    
x_folds, y_folds = k_fold_split(x_set, y_set, 5)

within k fold split function
x_set type is<class 'numpy.ndarray'>
y_set type is<class 'numpy.ndarray'>
X set is 60000
Y set is 60000
len of x final list 5
len of y final list 5
x_final_type <class 'list'>
y_final_type <class 'list'>
x_final_type[0] <class 'numpy.ndarray'>
y_final_type[0] <class 'list'>
length of y final[0] 12000
type of y final[0] <class 'list'>
x_final_type[0] shape (12000, 784)


In [3]:
def construct_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

#Epochs are the number of times the dataset will be iterated over, a good number is 20
def train_model(model, train_dataset, validation_dataset, epochs, name):
    x_set, y_set = train_dataset
    model.fit(x_set, y_set, epochs=epochs, batch_size=128, validation_data=validation_dataset)
    model.save(f'./{name}')

In [4]:
#Hint: Neural Networks can't just handle the lables as they are, they need --categorical-- data
#Note: You must submit the trained models along with the notebook for full credit
def train_validate_k(x_folds , y_folds, num_folds):
    '''
        Inputs: x_folds, the x folds returned from the k_fold algorithm above, 
        y_folds the y folds returned from the k_fold algorithm above
        num_folds, the number of folds used to make the x_folds and y_folds
        Expected Output: Nothing, this function has no explicit output, 
        but there must be num_fold models trained and saved to disk
    '''
    
    #use 10 as the size or else you get "expected dense to be array (10,) but got array (12,000,)". Resize to account for this
    size = 10
    
    for i in range(num_folds):
        model = construct_model()
    
        #create the train dataset
        x_train_dataset = np.reshape(x_folds[i], (-1, 784))
        y_train_dataset = to_categorical(y_folds[i], num_classes=size)
        train_dataset = (x_train_dataset, y_train_dataset)
        
        #make the next index be i+1 % num_folds to avoid index out of bounds error when doing test of 5 and validate at 0.
        
        #create the validate dataset
        x_validate_dataset = np.reshape(x_folds[((i+1)%num_folds)], (-1, 784))
        y_validate_dataset = to_categorical(y_folds[((i+1)%num_folds)], num_classes=size)
        validate_dataset = (x_validate_dataset, y_validate_dataset)
        
        #train the model
        train_model(model, train_dataset, validate_dataset, 20, 'Train_Model_'+str(i))


        
    

In [ ]:
x_folds = np.array(x_folds)
train_validate_k(x_folds, y_folds, 5)

Train on 12000 samples, validate on 12000 samples
Epoch 1/20
12000/12000 [==============================] - 3s 235us/step - loss: 0.4949 - acc: 0.8433 - val_loss: 0.2709 - val_acc: 0.9158
Epoch 2/20
12000/12000 [==============================] - 2s 208us/step - loss: 0.2180 - acc: 0.9339 - val_loss: 0.1755 - val_acc: 0.9451
Epoch 3/20
12000/12000 [==============================] - 2s 201us/step - loss: 0.1487 - acc: 0.9525 - val_loss: 0.2071 - val_acc: 0.9360
Epoch 4/20
12000/12000 [==============================] - 3s 231us/step - loss: 0.1047 - acc: 0.9664 - val_loss: 0.1669 - val_acc: 0.9494
Epoch 5/20
12000/12000 [==============================] - 3s 243us/step - loss: 0.0781 - acc: 0.9757 - val_loss: 0.1570 - val_acc: 0.9543
Epoch 6/20
12000/12000 [==============================] - 2s 184us/step - loss: 0.0603 - acc: 0.9796 - val_loss: 0.1283 - val_acc: 0.9627
Epoch 7/20
12000/12000 [==============================] - 3s 218us/step - loss: 0.0419 - acc: 0.9875 - val_loss: 0.1553 - 

12000/12000 [==============================] - 2s 172us/step - loss: 0.0134 - acc: 0.9960 - val_loss: 0.1873 - val_acc: 0.9673
Epoch 20/20
12000/12000 [==============================] - 2s 170us/step - loss: 0.0127 - acc: 0.9961 - val_loss: 0.1878 - val_acc: 0.9674
Train on 12000 samples, validate on 12000 samples
Epoch 1/20
12000/12000 [==============================] - 2s 199us/step - loss: 0.5137 - acc: 0.8394 - val_loss: 0.2999 - val_acc: 0.9104
Epoch 2/20
12000/12000 [==============================] - 2s 174us/step - loss: 0.1992 - acc: 0.9417 - val_loss: 0.2212 - val_acc: 0.9331
Epoch 3/20
12000/12000 [==============================] - 2s 174us/step - loss: 0.1356 - acc: 0.9573 - val_loss: 0.1971 - val_acc: 0.9417
Epoch 4/20
12000/12000 [==============================] - 2s 174us/step - loss: 0.1016 - acc: 0.9689 - val_loss: 0.2076 - val_acc: 0.9403
Epoch 5/20
12000/12000 [==============================] - 2s 175us/step - loss: 0.0715 - acc: 0.9774 - val_loss: 0.2113 - val_acc: 0

#### Section 3, write a few sentences below.